In [49]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble
from math import sqrt
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import BaggingClassifier 
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from mlxtend.classifier import StackingClassifier
#------------------------------------------------------------------------------------------
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
#----------------------------------------------------------------------------------------------------資料前處理
train_x = train[['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
Std = StandardScaler()
train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']] = Std.fit_transform(train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']]) #標準化
train_x[['HasCrCard','IsActiveMember']] = train_x[['HasCrCard','IsActiveMember']].astype(str)
train_x_dummies = pd.get_dummies(train_x)
#print(train_x_dummies)
train_y = train['Exited']
#------------------------------------------------------------------------------------------------------------------------------
test_x = test[['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']]
test_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']] = Std.fit_transform(test_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']]) #標準化
test_x[['HasCrCard','IsActiveMember']] = test_x[['HasCrCard','IsActiveMember']].astype(str)
test_x_dummies = pd.get_dummies(test_x)
dummy = test_x_dummies.columns.tolist()
#------------------------------------------------------------------------------------------
bagging = BaggingClassifier(n_estimators=80,max_samples=0.71, max_features=0.5)
hist = HistGradientBoostingClassifier(max_iter=80,learning_rate = 1, max_depth = 1) 
max_sample = [0.6,0.7,0.8]
max_feature = [round(sqrt(len(dummy))-1),round(sqrt(len(dummy))),round(sqrt(len(dummy))+1)]
forest = ensemble.RandomForestClassifier(criterion="entropy",random_state=10,max_samples=max_sample[0],max_features=max_feature[2])
c = [0.01,1,100,10000,1000000]
lr = LogisticRegression(C=c[2], penalty = "l2",random_state = 10)
alphas = [0.0001,0.01,1]
acti = ['identity', 'logistic', 'tanh', 'relu']
nn = MLPClassifier(alpha = alphas[2], random_state = 10, activation = acti[3],max_iter=1000)
c = [0.1,1,10]
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
svm = svm.SVC(C=c[1], random_state=10, kernel=kernels[1],probability=True)
sclf = StackingClassifier(classifiers=[forest,hist], meta_classifier=bagging,use_probas=True)
fit = sclf.fit(train_x_dummies, train_y)
pred_y = fit.predict(test_x_dummies)
predict = pred_y.tolist()
test['Exited'] = pred_y
# ----------------------------------------------------------------------------------------------------------------------
answer = {"RowNumber": test["RowNumber"], "Exited": test["Exited"]}
answer_df = pd.DataFrame(answer)
print(answer_df)
answer_df.to_csv("final.csv") 

<ipython-input-49-836977937f79>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']] = Std.fit_transform(train_x[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']]) #標準化
c:\python37\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
c:\python37\lib\site-packages\pandas\core\frame.py:3191: Sett

      RowNumber  Exited
0          2209       0
1          9924       0
2          4617       0
3          6077       0
4          9240       1
...         ...     ...
1995       7872       0
1996       4257       0
1997       2273       0
1998        315       0
1999       3628       0

[2000 rows x 2 columns]


In [37]:
train_x_dummies

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,HasCrCard_0,HasCrCard_1,IsActiveMember_0,IsActiveMember_1
0,1.509906,-0.663721,0.339933,0.428978,0.799546,0.304927,0,0,1,0,1,1,0,0,1
1,0.866637,1.465241,-0.445812,-1.214512,-0.917139,0.529787,1,0,0,1,0,0,1,1,0
2,0.697356,-1.595141,0.339933,0.615161,-0.917139,1.339115,1,0,0,1,0,0,1,0,1
3,-0.656894,-0.397600,-0.445812,0.117359,-0.917139,0.438752,0,0,1,1,0,0,1,1,0
4,2.068534,-0.131480,0.339933,0.630562,-0.917139,1.072015,0,0,1,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,-1.063169,-1.062901,1.125677,0.652575,0.799546,1.011302,1,0,0,1,0,0,1,0,1
7996,-0.487612,0.267700,1.911421,1.440568,-0.917139,-0.033763,0,1,0,1,0,0,1,1,0
7997,0.815853,0.001580,0.339933,1.025801,-0.917139,1.251627,0,0,1,1,0,0,1,1,0
7998,1.425266,-0.663721,-0.445812,0.577801,-0.917139,-0.188456,1,0,0,0,1,1,0,1,0


In [45]:
smo = SMOTE(k_neighbors = 5,n_jobs = -1,random_state=100)
train_x_dummies,train_y = SMOTE().fit_resample(train_x_dummies, train_y) 
train_x_dummies

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male,HasCrCard_0,HasCrCard_1,IsActiveMember_0,IsActiveMember_1
0,1.509906,-0.663721,0.339933,0.428978,0.799546,0.304927,0,0,1,0,1,1,0,0,1
1,0.866637,1.465241,-0.445812,-1.214512,-0.917139,0.529787,1,0,0,1,0,0,1,1,0
2,0.697356,-1.595141,0.339933,0.615161,-0.917139,1.339115,1,0,0,1,0,0,1,0,1
3,-0.656894,-0.397600,-0.445812,0.117359,-0.917139,0.438752,0,0,1,1,0,0,1,1,0
4,2.068534,-0.131480,0.339933,0.630562,-0.917139,1.072015,0,0,1,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12731,1.518171,0.520999,1.125677,-1.214512,-0.917139,-1.230433,0,0,1,0,1,0,0,1,0
12732,0.705542,1.204017,0.339933,-0.933968,-0.917139,1.332099,1,0,0,1,0,1,0,0,0
12733,-2.043085,0.282303,0.327614,-1.214512,2.516232,-0.300851,0,0,1,1,0,0,1,1,0
12734,-0.720391,-1.306816,-0.085722,0.396115,-0.917139,-1.128823,0,1,0,1,0,0,1,1,0
